In [1]:
import duckdb
import pandas as pd

con = duckdb.connect(database="ds_week1.duckdb")  # creates a local file DB
con.execute("SELECT 1").fetchall()


[(1,)]

In [2]:
import numpy as np
from datetime import datetime, timedelta

np.random.seed(42)

n_users = 2000
start = datetime(2025, 11, 1)
days = 30

# users
users = pd.DataFrame({
    "user_id": np.arange(1, n_users+1),
    "signup_date": [start + timedelta(days=int(x)) for x in np.random.randint(0, days, n_users)],
    "country": np.random.choice(["IN","US","GB","DE","BR"], n_users, p=[0.55,0.15,0.1,0.1,0.1]),
    "device": np.random.choice(["android","ios","web"], n_users, p=[0.5,0.35,0.15]),
})

# events
event_types = ["app_open","view_item","add_to_cart","checkout","purchase"]
rows = []
for uid in users["user_id"]:
    k = np.random.randint(5, 16)  # events per user
    for _ in range(k):
        day = int(np.random.randint(0, days))
        t = start + timedelta(days=day, minutes=int(np.random.randint(0, 24*60)))
        rows.append((uid, t, np.random.choice(event_types, p=[0.45,0.35,0.12,0.06,0.02]), int(np.random.randint(1, 4000))))
events = pd.DataFrame(rows, columns=["user_id","event_time","event_type","session_id"])

# orders (subset of purchase events)
purchase_events = events[events["event_type"] == "purchase"].copy()
purchase_events["order_id"] = np.arange(1, len(purchase_events)+1)
purchase_events["revenue"] = np.round(np.random.lognormal(mean=3.2, sigma=0.45, size=len(purchase_events)), 2)
orders = purchase_events.rename(columns={"event_time":"order_time"})[["order_id","user_id","order_time","revenue"]]

con.execute("DROP TABLE IF EXISTS users")
con.execute("DROP TABLE IF EXISTS events")
con.execute("DROP TABLE IF EXISTS orders")

con.register("users_df", users)
con.register("events_df", events)
con.register("orders_df", orders)

con.execute("CREATE TABLE users AS SELECT * FROM users_df")
con.execute("CREATE TABLE events AS SELECT * FROM events_df")
con.execute("CREATE TABLE orders AS SELECT * FROM orders_df")

con.execute("SELECT COUNT(*) FROM users").fetchone(), con.execute("SELECT COUNT(*) FROM events").fetchone(), con.execute("SELECT COUNT(*) FROM orders").fetchone()


((2000,), (20002,), (402,))

In [3]:
con.close()
